In [3]:
import os
import logging
import glob
import shutil

import numpy as np
import tensorflow as tf

from tensorflow.keras import layers

In [4]:
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print("TensorFlow version: ", tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
gpus

TensorFlow version:  2.16.1
1 Physical GPUs, 1 Logical GPUs


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
data_path = "../data"


classes = ['matmekh',
            'Shayba',
            'Dvadcatki',
            'Desyatki',
            'pmpu'
]
num_classes = len(classes)
classes

['matmekh', 'Shayba', 'Dvadcatki', 'Desyatki', 'pmpu']

In [6]:
for cl in classes:
    img_path = os.path.join(data_path, cl)
    images = glob.glob(img_path + '/*.jpg')
    print(f"{cl} : {len(images)} Images")
    train, val = images[:round(len(images)*0.8)], images[round(len(images)*0.8):]

    for t in train:
        if not os.path.exists(os.path.join(data_path, 'train', cl)):
            os.makedirs(os.path.join(data_path, 'train', cl))
        shutil.move(t, os.path.join(data_path, 'train', cl))

    for v in val:
        if not os.path.exists(os.path.join(data_path, 'val', cl)):
            os.makedirs(os.path.join(data_path, 'val', cl))
        shutil.move(v, os.path.join(data_path, 'val', cl))

matmekh : 0 Images
Shayba : 0 Images
Dvadcatki : 0 Images
Desyatki : 0 Images
pmpu : 0 Images


In [7]:
train_dir = os.path.join(data_path, "train")
val_dir = os.path.join(data_path, "val")

total_train = sum(len(os.listdir(os.path.join(train_dir, cl))) for cl in classes)
total_val = sum(len(os.listdir(os.path.join(val_dir, cl))) for cl in classes)


In [8]:
BATCH_SIZE = 16
IMG_SHAPE = 160

image_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, rotation_range=45, width_shift_range=0.15,
    height_shift_range=0.15, vertical_flip=True, horizontal_flip=True,
    zoom_range=0.4, fill_mode='nearest'
)

image_gen_val = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
)

train_data_gen = image_gen.flow_from_directory(
    batch_size=BATCH_SIZE, directory=train_dir, shuffle=True,
    target_size=(IMG_SHAPE, IMG_SHAPE), class_mode='sparse'
)

val_data_gen = image_gen_val.flow_from_directory(
    batch_size=BATCH_SIZE, directory=val_dir, target_size=(IMG_SHAPE, IMG_SHAPE),
    class_mode='sparse'
)


Found 1326 images belonging to 5 classes.
Found 331 images belonging to 5 classes.


In [9]:
model = tf.keras.models.Sequential([
    layers.InputLayer(shape=(IMG_SHAPE,IMG_SHAPE,3)),
    layers.Conv2D(128, (5, 5), padding="same", activation='relu'),
    layers.Conv2D(128, (3, 3), strides=2, padding="same", activation='relu'),
    layers.Conv2D(64, (5, 5), padding="same", activation='relu'),
    layers.Conv2D(64, (3, 3), strides=2, padding="same", activation='relu'),
    layers.Conv2D(32, (5, 5), padding="same", activation='relu'),
    layers.Conv2D(32, (3, 3), strides=2, padding="same", activation='relu'),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 160, 160, 128)  │         9,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 80, 80, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 80, 80, 64)     │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 40, 40, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 40, 40, 32)     │        51,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 20, 20, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,638,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,108,613 (8.04 MB)

 Trainable params: 2,108,613 (8.04 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [12]:
checkpoint_path = "../trained_models/ckpt/checkpoint.model.keras"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, verbose=1)

In [13]:
EPOCHS = 50


history = model.fit(
    train_data_gen, validation_data=val_data_gen, batch_size=BATCH_SIZE,
    epochs=EPOCHS, steps_per_epoch=int(np.ceil(total_train / BATCH_SIZE)),
    validation_steps=int(np.ceil(total_val / BATCH_SIZE)), callbacks=[cp_callback]
)

Epoch 1/50


/home/punky/Coding/projects/PUNK-Places-Recognition-Project/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1714409794.164059    4848 service.cc:145] XLA service 0x7aa61c0114c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714409794.164082    4848 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2024-04-29 19:56:34.199426: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 19:56:34.346365: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:

: 